In [119]:
import os, sys, json
import numpy as np
import pandas as pd
from typing import List
import re
import ast
from pprint import pprint

def move_working_dir_to_repo_root(repo_name="orgsync"):
    """
    Move the current working directory to the root of the repository.
    """
    current_dir = os.getcwd()
    while os.path.basename(current_dir).lower() != repo_name:
        current_dir = os.path.dirname(current_dir)
    os.chdir(current_dir)
    print("Current working directory: ", os.getcwd())

move_working_dir_to_repo_root(repo_name="orgsync")
# All unique postcodes in GtR associated one or more organisation_ids
data_dir = "data/geolocations/orgs_names_postcodes_grouped.csv"
# All unique postcodes in GtR associated with more than one organisation_id
# data_dir = "data/geolocations/orgs_names_postcodes_grouped_unique_only.csv"
gtr_organisations = "data/geolocations/gtr_organizations.csv"
df = pd.read_csv(data_dir)

# convert org_ids values to list, currently string representation of list
df["org_ids"] = df["org_ids"].apply(lambda x: ast.literal_eval(x))
df["num_ids"] = df["org_ids"].apply(lambda x: len(x))
# df = df.sort_values("num_ids", ascending=False).reset_index(drop=True)
# # check types of org_ids column by printing all unique types 
print(set([type(x) for x in df["org_ids"]]))


df.head()

Current working directory:  c:\Users\dec2g\GitHub\OrgSync
{<class 'list'>}


,postcode,org_ids,num_ids
0,A9 2VW,[6B678B02-3DAB-41F1-BA5E-54B510304C7F],1
1,A9 8HF,[F971E743-BECD-4A6A-B547-8E26C6F7C2EB],1
2,AA1 1AA,[B439F284-54EB-4933-BA31-3797FBE211CC],1
3,AB10 1AB,"[7ED73A50-A205-49E0-A635-2CB09B1555E3, 4BD11E7...",3
4,AB10 1BL,[825A0B07-A723-4AFE-BF05-03B79719FA6E],1


In [120]:
df_exploded = df.explode("org_ids").reset_index(drop=True)
df_exploded.head(50)

,postcode,org_ids,num_ids
0,A9 2VW,6B678B02-3DAB-41F1-BA5E-54B510304C7F,1
1,A9 8HF,F971E743-BECD-4A6A-B547-8E26C6F7C2EB,1
2,AA1 1AA,B439F284-54EB-4933-BA31-3797FBE211CC,1
3,AB10 1AB,7ED73A50-A205-49E0-A635-2CB09B1555E3,3
4,AB10 1AB,4BD11E7C-6951-4E2A-9BEC-BEBB64472F95,3
5,AB10 1AB,B783D209-E918-4E4D-AB44-89B1A0FF6817,3
6,AB10 1BL,825A0B07-A723-4AFE-BF05-03B79719FA6E,1
7,AB10 1DQ,0EB554AB-6EC9-4283-ACF2-28165A3D5CA8,1
8,AB10 1FE,77944907-3066-483B-A141-EB76BCBB0814,1
9,AB10 1FQ,F8C26AE4-AC08-4020-BA02-111087D5AC4F,1


In [129]:
# df_by_postcode = df_exploded.groupby("postcode").agg({"org_ids": list}).reset_index()
df_by_postcode = df_exploded.groupby("postcode")["org_ids"].unique().reset_index()
df_by_postcode.head(12)

,postcode,org_ids
0,A9 2VW,[6B678B02-3DAB-41F1-BA5E-54B510304C7F]
1,A9 8HF,[F971E743-BECD-4A6A-B547-8E26C6F7C2EB]
2,AA1 1AA,[B439F284-54EB-4933-BA31-3797FBE211CC]
3,AB10 1AB,"[7ED73A50-A205-49E0-A635-2CB09B1555E3, 4BD11E7..."
4,AB10 1BL,[825A0B07-A723-4AFE-BF05-03B79719FA6E]
5,AB10 1DQ,[0EB554AB-6EC9-4283-ACF2-28165A3D5CA8]
6,AB10 1FE,[77944907-3066-483B-A141-EB76BCBB0814]
7,AB10 1FQ,[F8C26AE4-AC08-4020-BA02-111087D5AC4F]
8,AB10 1FR,"[2C2EF413-6B81-4F48-BE30-58F0BF3291F0, D2CEE12..."
9,AB10 1FW,"[196C380E-218E-4E8B-89DA-1B84D05AD536, C1FBF68..."


In [128]:
df_by_id = df_exploded.copy().reset_index().groupby("org_ids")["postcode"].unique().reset_index()
df_by_id = df_by_id.sort_values("postcode", ascending=True)
df_by_id.head(12)

,org_ids,postcode
15088,6B678B02-3DAB-41F1-BA5E-54B510304C7F,[A9 2VW]
35067,F971E743-BECD-4A6A-B547-8E26C6F7C2EB,[A9 8HF]
25453,B439F284-54EB-4933-BA31-3797FBE211CC,[AA1 1AA]
10658,4BD11E7C-6951-4E2A-9BEC-BEBB64472F95,[AB10 1AB]
17773,7ED73A50-A205-49E0-A635-2CB09B1555E3,[AB10 1AB]
25909,B783D209-E918-4E4D-AB44-89B1A0FF6817,[AB10 1AB]
18276,825A0B07-A723-4AFE-BF05-03B79719FA6E,[AB10 1BL]
2092,0EB554AB-6EC9-4283-ACF2-28165A3D5CA8,[AB10 1DQ]
16766,77944907-3066-483B-A141-EB76BCBB0814,[AB10 1FE]
34962,F8C26AE4-AC08-4020-BA02-111087D5AC4F,[AB10 1FQ]


In [132]:
# print any ids that occur in more than one row
df_by_id["num_postcodes"] = df_by_id["postcode"].apply(lambda x: len(x))
# sort by number of postcodes
df_by_id = df_by_id.sort_values("num_postcodes", ascending=True)
df_by_id.head()

,org_ids,postcode,num_postcodes
18276,825A0B07-A723-4AFE-BF05-03B79719FA6E,[AB10 1BL],1
2092,0EB554AB-6EC9-4283-ACF2-28165A3D5CA8,[AB10 1DQ],1
16766,77944907-3066-483B-A141-EB76BCBB0814,[AB10 1FE],1
34962,F8C26AE4-AC08-4020-BA02-111087D5AC4F,[AB10 1FQ],1
29646,D2CEE122-2B5E-4F0E-BB2F-C97467084E27,[AB10 1FR],1


In [135]:
# get column org_ids as list
org_ids = df_by_id["org_ids"].tolist()
# do any ids occur more than once?
seen = set()
duplicates = set()
duplicate_ids = []
for i in org_ids:

    if i in seen:
        duplicates.add(i)
        if i not in duplicate_ids:
            duplicate_ids.append(i)
    else:
        seen.add(i)
print("Number of duplicate ids: ", len(duplicates))
print("Number of unique ids: ", len(seen))
print("dupliate ids", duplicate_ids)

Number of duplicate ids:  0
Number of unique ids:  35971
dupliate ids []


the above might be working, not sure actually - we may just have a case where each id is only associated with one postcode.

In [115]:
df_by_id = df_exploded.groupby("org_ids").agg({"postcode": list}).reset_index()
df_by_id.head()
df_by_id = df_by_id.sort_values("postcode", ascending=True)
df_by_id.head(50)

,org_ids,postcode
15088,6B678B02-3DAB-41F1-BA5E-54B510304C7F,[A9 2VW]
35067,F971E743-BECD-4A6A-B547-8E26C6F7C2EB,[A9 8HF]
25453,B439F284-54EB-4933-BA31-3797FBE211CC,[AA1 1AA]
10658,4BD11E7C-6951-4E2A-9BEC-BEBB64472F95,[AB10 1AB]
17773,7ED73A50-A205-49E0-A635-2CB09B1555E3,[AB10 1AB]
25909,B783D209-E918-4E4D-AB44-89B1A0FF6817,[AB10 1AB]
18276,825A0B07-A723-4AFE-BF05-03B79719FA6E,[AB10 1BL]
2092,0EB554AB-6EC9-4283-ACF2-28165A3D5CA8,[AB10 1DQ]
16766,77944907-3066-483B-A141-EB76BCBB0814,[AB10 1FE]
34962,F8C26AE4-AC08-4020-BA02-111087D5AC4F,[AB10 1FQ]


groupby not aggregating to list - need to fix

In [111]:
df_by_id = df_exploded.groupby("postcode")["org_ids"].agg(list).reset_index()
df_by_id.head()

,postcode,org_ids
0,A9 2VW,[6B678B02-3DAB-41F1-BA5E-54B510304C7F]
1,A9 8HF,[F971E743-BECD-4A6A-B547-8E26C6F7C2EB]
2,AA1 1AA,[B439F284-54EB-4933-BA31-3797FBE211CC]
3,AB10 1AB,"[7ED73A50-A205-49E0-A635-2CB09B1555E3, 4BD11E7..."
4,AB10 1BL,[825A0B07-A723-4AFE-BF05-03B79719FA6E]


In [112]:
df_by_id.head(50)
# sort alphabetically by column postcode
df_by_id = df_by_id.sort_values("postcode", ascending=True)
df_by_id.head(50)

,postcode,org_ids
0,A9 2VW,[6B678B02-3DAB-41F1-BA5E-54B510304C7F]
1,A9 8HF,[F971E743-BECD-4A6A-B547-8E26C6F7C2EB]
2,AA1 1AA,[B439F284-54EB-4933-BA31-3797FBE211CC]
3,AB10 1AB,"[7ED73A50-A205-49E0-A635-2CB09B1555E3, 4BD11E7..."
4,AB10 1BL,[825A0B07-A723-4AFE-BF05-03B79719FA6E]
5,AB10 1DQ,[0EB554AB-6EC9-4283-ACF2-28165A3D5CA8]
6,AB10 1FE,[77944907-3066-483B-A141-EB76BCBB0814]
7,AB10 1FQ,[F8C26AE4-AC08-4020-BA02-111087D5AC4F]
8,AB10 1FR,"[2C2EF413-6B81-4F48-BE30-58F0BF3291F0, D2CEE12..."
9,AB10 1FW,"[196C380E-218E-4E8B-89DA-1B84D05AD536, C1FBF68..."


In [100]:
df_by_id["num_postcodes"] = df_by_id["postcode"].apply(lambda x: len(x))
df_by_id = df_by_id.sort_values("num_postcodes", ascending=True).reset_index(drop=True)
df_by_id.head()

,org_ids,postcode,num_postcodes
0,FFFDCB8B-EF0F-49FF-81FD-73FBF10C4635,[PE9 2PF],1
1,FFFC644A-7159-4318-9464-E17C504CC75C,[BN2 5BE],1
2,FFFA7DAA-649F-454C-AD97-11B6FFFF98ED,[LA11 6PG],1
3,FFFA73D6-BFE8-450E-BBDA-C13A584D5E8B,[EC4A 4AB],1
4,FFF4E233-AF8E-432D-8AB1-FDC2104438E8,[HG4 3JB],1


In [101]:
df_by_postcode = df_by_id.explode("postcode")
df_by_postcode.head(50)

,org_ids,postcode,num_postcodes
0,FFFDCB8B-EF0F-49FF-81FD-73FBF10C4635,PE9 2PF,1
1,FFFC644A-7159-4318-9464-E17C504CC75C,BN2 5BE,1
2,FFFA7DAA-649F-454C-AD97-11B6FFFF98ED,LA11 6PG,1
3,FFFA73D6-BFE8-450E-BBDA-C13A584D5E8B,EC4A 4AB,1
4,FFF4E233-AF8E-432D-8AB1-FDC2104438E8,HG4 3JB,1
5,FFF41672-78D5-4DCD-96B4-92AF15F5256C,BT7 2JD,1
6,FFF38A82-6EC7-416E-AC9B-65F146A383A7,B1 1TT,1
7,FFF3861F-42BC-4C20-8282-0E5E2B190B3D,L1 0AF,1
8,FFF1D855-1FEC-4297-AE50-F12E71F50FF9,DE56 4AN,1
9,FFED646F-96A2-4C22-B768-8EBF15863175,TA3 7NN,1


In [102]:
df_by_postcode.groupby("postcode")["org_ids"].agg(list).reset_index()
df_by_postcode.head(50)
# sort alphabetically by postcode
df_by_postcode = df_by_postcode.sort_values("postcode", ascending=True).reset_index(drop=True)
df_by_postcode.head(50)

,org_ids,postcode,num_postcodes
0,6B678B02-3DAB-41F1-BA5E-54B510304C7F,A9 2VW,1
1,F971E743-BECD-4A6A-B547-8E26C6F7C2EB,A9 8HF,1
2,B439F284-54EB-4933-BA31-3797FBE211CC,AA1 1AA,1
3,7ED73A50-A205-49E0-A635-2CB09B1555E3,AB10 1AB,1
4,4BD11E7C-6951-4E2A-9BEC-BEBB64472F95,AB10 1AB,1
5,B783D209-E918-4E4D-AB44-89B1A0FF6817,AB10 1AB,1
6,825A0B07-A723-4AFE-BF05-03B79719FA6E,AB10 1BL,1
7,0EB554AB-6EC9-4283-ACF2-28165A3D5CA8,AB10 1DQ,1
8,77944907-3066-483B-A141-EB76BCBB0814,AB10 1FE,1
9,F8C26AE4-AC08-4020-BA02-111087D5AC4F,AB10 1FQ,1


In [70]:
## TO FIX
# def postcode_to_orgid(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Convert dataframe from unique postcodes with lists of org_ids
#     to unique org_ids with lists of postcodes.
#     """
#     # if num_ids column present, drop it
#     if "num_ids" in df.columns:
#         df = df.drop(columns=["num_ids"])
#     # Explode the org_ids list to create a row for each org_id
#     exploded_df = df.explode('org_ids')
    
#     # Group by org_id and aggregate postcodes into a list
#     result_df = exploded_df.groupby('org_ids')['postcode'].agg(list).reset_index()
    
#     # Rename columns for clarity
#     result_df.columns = ['org_id', 'postcodes']
#     result_df["num_postcodes"] = result_df["postcodes"].apply(lambda x: len(x))
    
#     return result_df

# def orgid_to_postcode(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Convert dataframe from unique org_ids with lists of postcodes
#     to unique postcodes with lists of org_ids.
#     """
#     if "num_postcodes" in df.columns:
#         df = df.drop(columns=["num_postcodes"])
#     # Explode the postcodes list to create a row for each postcode
#     exploded_df = df.explode('postcodes')
    
#     # Group by postcode and aggregate org_ids into a list
#     result_df = exploded_df.groupby('postcodes')['org_id'].agg(list).reset_index()
    
#     # Rename columns for clarity
#     result_df.columns = ['postcode', 'org_ids']
#     result_df["num_ids"] = result_df["org_ids"].apply(lambda x: len(x))
#     return result_df


# df_orgid = postcode_to_orgid(df)
# df_postcode = orgid_to_postcode(df_orgid)
# # check if the conversion is correct
# if not df.equals(df_postcode):
#     print("Conversation is incorrect")
# # df_orgid.head()
# df_postcode.sort_values("num_ids", ascending=False)
# df_postcode.head()


Conversation is incorrect


,postcode,org_ids,num_ids
0,AB10 1AB,"[4BD11E7C-6951-4E2A-9BEC-BEBB64472F95, 7ED73A5...",3
1,AB10 1FR,"[2C2EF413-6B81-4F48-BE30-58F0BF3291F0, D2CEE12...",2
2,AB10 1FW,"[196C380E-218E-4E8B-89DA-1B84D05AD536, C1FBF68...",2
3,AB10 1HA,"[7D77BAEB-5581-4B9F-AC76-54AD5A9EDBE0, 90D8464...",5
4,AB10 1UP,"[18C433C7-8322-43E2-B415-23B0517AC1A6, 2C81420...",2


Note: the data used here assumes we haven't matched any org names yet. In practice, we could want to do this first, then look at any postcodes with more than one org ID. 

We have already removed postcodes from our dataset associated with only 1 organisation id, but we might want to get this back for. 

In [46]:
df.head()

,postcode,org_ids,num_ids
22058,WC2H 9JQ,"['010D8FA8-E2B1-4BB1-9FAE-354D039F7DF4', 'A08C...",168
6373,EC1V 2NX,"['5370084A-BDBF-4A7B-8E02-09B7DE95D01B', 'F911...",121
12531,N1 7GU,"['5B52B7CA-6400-4222-8F37-0AB89D39892A', '3ADA...",98
3501,CB4 0WS,"['C01F63F1-67FB-4E88-8508-0565D666403C', '9814...",68
6529,EC2A 4NE,"['575F14A8-F64F-4242-B1DC-4E98E5B5E95D', '11C2...",64


In [47]:
df["num_ids"] = df["org_ids"].apply(lambda x: len(ast.literal_eval(x)))
df = df.sort_values("num_ids", ascending=False)
df.head()



,postcode,org_ids,num_ids
22058,WC2H 9JQ,"['010D8FA8-E2B1-4BB1-9FAE-354D039F7DF4', 'A08C...",168
6373,EC1V 2NX,"['5370084A-BDBF-4A7B-8E02-09B7DE95D01B', 'F911...",121
12531,N1 7GU,"['5B52B7CA-6400-4222-8F37-0AB89D39892A', '3ADA...",98
3501,CB4 0WS,"['C01F63F1-67FB-4E88-8508-0565D666403C', '9814...",68
6529,EC2A 4NE,"['575F14A8-F64F-4242-B1DC-4E98E5B5E95D', '11C2...",64


In [48]:
# get first row and get value of "org_ids" as a list
postcode_w_most_orgids = df.iloc[0]["org_ids"]

print("Postcode with most organisation_ids: ", postcode_w_most_orgids)

Postcode with most organisation_ids:  ['010D8FA8-E2B1-4BB1-9FAE-354D039F7DF4', 'A08CFD3F-B7A7-48B0-89A4-37CB1AC36DC0', 'C2A9D760-A013-4D9F-BC73-1CB8FA7C31CD', 'F039564E-C056-4751-B740-7E8C7989AF4A', 'C7490FBD-B738-4547-8579-2F929D0286A9', '29FBBB9E-7AF1-4CFB-8B9D-9C20FC25F935', '2EB658DE-55A2-4A9B-B1C0-A01E80412D22', '7F62A7CB-B3EA-426C-98E1-821A3F5ACFB6', '991BB18A-178F-4E7B-856A-A64B31690901', 'AD859484-F473-4A29-B4E5-396EA60A8B11', '03F2BE6D-5A43-41BB-8602-4D882BFB3EC2', 'F1AC24D9-E6E4-450B-A018-00B23A48EBA6', '9B5219D7-A3B2-416B-833E-10F444BB4835', 'B4DF6D9D-25DD-404F-B419-0440DD1AFAEE', '70A8849D-DFE4-46D8-B294-2138113B65E7', '95785693-3A1F-4A40-92DA-192B434205C7', '9881201D-036E-475D-A1C3-2E87401EEFC0', '21324F60-F4DF-4EF0-A7FB-20EE0D7A4BD9', '93005DA3-4607-40AC-9EBE-0C477C01C5CC', '34ED69C6-FEE5-4C25-8199-552611BD9353', 'FF8E372D-16BF-4B25-9E05-0C0DB13E1EED', 'AFA52DFE-B8F0-4A26-B213-6A419436D873', '871B4322-579A-47C7-BCEB-23C517C3082B', '195E005B-F0B6-4946-BCBB-00DBDE43D040', '

In [49]:
# check that all organisation_ids are unique
org_ids = ast.literal_eval(postcode_w_most_orgids)
# get_num_unique
num_unique_org_ids = len(set(org_ids))
print("Number of unique organisation_ids: ", num_unique_org_ids)

Number of unique organisation_ids:  168


In [50]:
orgs_df = pd.read_csv(gtr_organisations)
orgs_df.head()

,dataset,id,name,postcode
0,gtr,5331B126-3AB4-4412-B56D-00E8F2796556,NEWCASTLE CITY COUNCIL,NE1 8QH
1,gtr,53331120-0290-49FA-A513-0286A214AF7A,VALERANN UK LIMITED,MK14 6GD
2,gtr,77874202-2018-4677-8CFF-0868CD838659,Baltic Sea Cultural Centre in Gdansk,NaN
3,gtr,77908BF8-1B2D-4D26-9119-155100E8B9C5,Mindray,PE29 6FN
4,gtr,7794C645-9CC7-4913-A8DC-103AE0EFDD4B,Democracy International,20814


1.  Find org ids that are in more than one row (e.g. associated with more than one postcode)

We have a df with cols
* postcode: a unique postcode
* org_ids: a list of org_ids associated with that postcode

To find org_ids that are in more than one row, we can:
* explode the org_ids list into separate rows
* group by org_id and count the number of rows
* filter for rows where the count is greater than 1
* join back to the original df to get the postcodes associated with these org_ids

In [51]:
# find org_ids in that are in more than one row (e.g. have more than one postcode)
